In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

# Load the dataset
file_path = '33/data.csv'
df = pd.read_csv(file_path)

# Ensure the date format is correct
df['Date'] = pd.to_datetime(df['Date'])

# Group by Genre
genres = df['Genre'].unique()

# Parameter settings
look_back = 12  # Use data from the past 12 months for prediction
future_steps = 1  # Predict for the next 1 month

# Function to create time-series data
def create_time_series(data, look_back, future_steps):
    X, y = [], []
    for i in range(len(data) - look_back - future_steps + 1):
        X.append(data[i:i + look_back])
        y.append(data[i + look_back:i + look_back + future_steps])
    return np.array(X), np.array(y)

# Store model prediction results
results = {}

# Iterate through each Genre for prediction
for genre in genres:
    genre_data = df[df['Genre'] == genre].sort_values('Date')
    points = genre_data['Points (Total)'].values

    # Create time-series data
    X, y = create_time_series(points, look_back, future_steps)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

    # Reshape data to fit the LSTM model
    X_train = np.expand_dims(X_train, axis=-1)
    X_test = np.expand_dims(X_test, axis=-1)
    y_train = np.squeeze(y_train)
    y_test = np.squeeze(y_test)

    # Build the LSTM model
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(64, input_shape=(look_back, 1), return_sequences=False),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(future_steps)
    ])

    model.compile(optimizer='adam', loss='mse')

    # Train the model
    history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_split=0.1, verbose=1)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluation metrics
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    # Save results
    results[genre] = {
        'MSE': mse,
        'MAE': mae,
        'R2': r2,
        'Predictions': y_pred,
        'Actual': y_test
    }

    print(f"Genre {genre}: MSE={mse:.4f}, MAE={mae:.4f}, R2={r2:.4f}")

    # Plot the comparison of actual and predicted values
    plt.figure(figsize=(10, 6))
    plt.plot(y_test, label='Actual', color='blue')
    plt.plot(y_pred, label='Predicted', color='orange')
    plt.title(f"Genre {genre} - Actual vs Predicted Points")
    plt.xlabel("Time")
    plt.ylabel("Points (Total)")
    plt.legend()
    plt.show()
